In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random, math, time, os, datetime
import importlib

In [25]:
# Run this cell to load/reload modules:
os.chdir("../scripts")

import BTC_KnapFallBack
import BTC_KnapAlg
import BTC_KnapLevAlg
import BTC_Simulation
import BTC_Utils
importlib.reload(BTC_KnapFallBack)
importlib.reload(BTC_KnapAlg)
importlib.reload(BTC_KnapLevAlg)
importlib.reload(BTC_Simulation)
importlib.reload(BTC_Simulation)
importlib.reload(BTC_Utils)
from BTC_KnapFallBack import *
from BTC_KnapAlg import *
from BTC_KnapLevAlg import *
from BTC_Simulation import * 
from BTC_Utils import * 

os.chdir("../notebooks")

In [26]:
# Load UTXO and Payment Request Data

df_utxo = pd.read_csv("../data/utxo_sample.csv")
df_cc = pd.read_csv("../data/cc_data.csv")
cc_amts = df_cc['TransactionAmt'].apply(lambda x: USD_to_Satoshi(x,8582)) # Convert USD to Satoshi
cc_amts = cc_amts[cc_amts > 4004] # Ignore amounts <= 4004 in Satoshi

df_utxo = df_utxo[df_utxo['amount'] >= 4004] # Only consider UTXOs larger than 4004 Satoshi
df_pr = df_utxo[df_utxo['amount'] <= df_utxo['amount'].quantile(.95)] # Consider "resonable" UTXOs as payment requests

In [27]:
# Random Sample UTXO data

def RandomSampleUTXOData(num_utxos, num_pay_req, num_high_urg, randomState, payRequestData = 'CC'):
    
    if payRequestData == 'UTXO':
        utxos = np.array(df_utxo['amount'].sample(num_utxos, random_state=randomState))
        pay_req = np.array(df_pr['amount'].sample(num_pay_req, random_state=randomState))
        high_urg = pay_req[:num_high_urg]
    else:
        utxos = np.array(df_utxo['amount'].sample(num_utxos, random_state=randomState))
        pay_req = np.array(cc_amts.sample(num_pay_req, random_state=randomState))
        high_urg = pay_req[:num_high_urg]
    
    return utxos, pay_req, high_urg

In [ ]:
knap_sr = pd.read_csv("../simulation-results/KnapsackSuccessRate_noLev.csv") # Results of prior simulations to get Boost Factor

In [ ]:
def GetBoostFactor(feePerByte, payRequestsPerTX, df):
    return 1- df[df['feePerByte'] == feePerByte][df['Pay Requests Per TX'] == payRequestsPerTX]['Knapsack Success Rate'].iloc[0]

In [41]:
sims = []
summaries = []

startTimeFull = time.time()

# parameters

numUTXOs = 2500 # Size of UTXO Pool
numPayRequests = 250 # Size of Payment Requests Pool

for numHighUrg in [2,3,5,10]: # Range of values to test for number of pay reqs to process each tx
    for feePerByte in [22, 60, 200, 400, 900]: # Range of values to test fee-per-byte
        for rand in [17, 19, 21, 23, 31, 32, 33, 34, 35, 71]: # Range of random seeds for simulation
            utxos, pay_req, high_urg = RandomSampleUTXOData(numUTXOs, numPayRequests, numHighUrg, rand, payRequestData="CC")
            startTime = time.time()
            sim, summary = BTC_Simulation_KnapLev(utxos, pay_req, numHighUrg, leverage=False, print_details=True, iterations=5, \
                                   fee_per_byte = feePerByte, oneBTC = 8582, \
                                   lev_t1lb = numHighUrg, lev_t1ub = numHighUrg, lev_boost_factor = GetBoostFactor(feePerByte, numHighUrg,knap_sr), \
                                     time_limit_knap = 10)
            sims.append(sim)
            summaries.append(summary)

endTimeFull = time.time()

C:\Users\daniel.diroff\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


/////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////
Iteration : 1
UTXO Pool Size: 2500
Pay Requests Left: 250
/////////////////////////////////////////////////////////////////////
Trying Knapsack Solution:
Problem Status: Infeasible
The problem took 0.30472612380981445 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $1201.4267003975274
Payment Requests Processed:
p_0: Value = $97.0
p_1: Value = $1104.0
Output Total: $1201.0
Transaction Fee: $0.42669703999999997
Overpayment: $3.3575274373590945e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasib

Problem Status: Infeasible
The problem took 0.3515665531158447 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $374.4266960133927
Payment Requests Processed:
p_0: Value = $59.00000000000001
p_1: Value = $315.0
Output Total: $373.99999999999994
Transaction Fee: $0.42669703999999997
Overpayment: $-1.026607317775488e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment Less than Dust: True
Valid Transaction: False
Fall Back Solution: True
/////////////////////////////////////////////////////////////////////
Using Fallback Solution
/////////////////////

Problem Status: Infeasible
The problem took 0.24982500076293945 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $75.37669702006637
Payment Requests Processed:
p_0: Value = $49.0
p_1: Value = $25.950000000000003
Output Total: $74.95
Transaction Fee: $0.42669703999999997
Overpayment: $-1.993363012094051e-08
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment Less than Dust: True
Valid Transaction: False
Fall Back Solution: True
/////////////////////////////////////////////////////////////////////
Using Fallback Solution
/////////////////////////////////

Problem Status: Infeasible
The problem took 0.3593161106109619 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $339.4266971029469
Payment Requests Processed:
p_0: Value = $59.00000000000001
p_1: Value = $280.00000000000006
Output Total: $339.0
Transaction Fee: $0.42669703999999997
Overpayment: $6.294684173539281e-08
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: True
Overpayment Less than Dust: True
Valid Transaction: False
Fall Back Solution: True
/////////////////////////////////////////////////////////////////////
Using Fallback Solution
///////////////////////

...

Problem Status: Infeasible
The problem took 0.577279806137085 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $1102.6295217525071
Payment Requests Processed:
p_0: Value = $62.61500000000001
p_1: Value = $29.000000000000007
p_2: Value = $120.0
p_3: Value = $171.0
p_4: Value = $390.00000000000006
p_5: Value = $77.95
p_6: Value = $53.97
p_7: Value = $59.63
p_8: Value = $41.0
p_9: Value = $59.00000000000001
Output Total: $1064.1650000000002
Transaction Fee: $38.464524
Overpayment: $-2.2474932687869747e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpaymen

Problem Status: Infeasible
The problem took 0.8201992511749268 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $737.0905186212372
Payment Requests Processed:
p_0: Value = $29.000000000000007
p_1: Value = $49.0
p_2: Value = $100.00000000000003
p_3: Value = $20.0
p_4: Value = $100.00000000000003
p_5: Value = $57.95
p_6: Value = $42.676
p_7: Value = $151.0
p_8: Value = $49.0
p_9: Value = $100.00000000000003
Output Total: $698.6260000000001
Transaction Fee: $38.464524
Overpayment: $-5.378762999549508e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment

Problem Status: Infeasible
The problem took 0.7539975643157959 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $1041.080522634865
Payment Requests Processed:
p_0: Value = $82.95
p_1: Value = $92.805
p_2: Value = $226.0
p_3: Value = $57.00000000000001
p_4: Value = $186.00000000000003
p_5: Value = $82.0
p_6: Value = $20.95
p_7: Value = $29.441000000000003
p_8: Value = $54.470000000000006
p_9: Value = $171.0
Output Total: $1002.616
Transaction Fee: $38.464524
Overpayment: $-1.3651350478641689e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment Less t

Problem Status: Infeasible
The problem took 0.6886963844299316 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $1174.734519167456
Payment Requests Processed:
p_0: Value = $57.95
p_1: Value = $104.47
p_2: Value = $97.0
p_3: Value = $167.95
p_4: Value = $57.95
p_5: Value = $25.000000000000007
p_6: Value = $470.0
p_7: Value = $67.95
p_8: Value = $59.00000000000001
p_9: Value = $29.000000000000007
Output Total: $1136.2700000000002
Transaction Fee: $38.464524
Overpayment: $-4.832544135762168e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment Less than

Problem Status: Infeasible
The problem took 0.5852739810943604 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $919.6315401294634
Payment Requests Processed:
p_0: Value = $125.00000000000001
p_1: Value = $117.0
p_2: Value = $54.0
p_3: Value = $175.0
p_4: Value = $13.403
p_5: Value = $75.0
p_6: Value = $107.95
p_7: Value = $98.364
p_8: Value = $46.50000000000001
p_9: Value = $68.95
Output Total: $881.1670000000001
Transaction Fee: $38.464524
Overpayment: $1.612946329880695e-05
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: True
Overpayment Less than Dust: True
Vali

In [42]:
print("The simulations took: " + str(endTimeFull - startTimeFull) + " seconds.")
print("The simulations took: " + str((endTimeFull - startTimeFull)/60) + " minutes.")

The simulations took: 658.427985906601 seconds.


In [ ]:
save_results_path = "../simulation-results/no-leverage-results.csv" # path where to save csv file with simulation results

results = pd.DataFrame(summaries)
results.to_csv(save_results_path, index=False)